# Orbit Propagation Software Demo with Antoine The Intern

This tutorial demonstrates how to perform orbit propagation using the propagated orbit mode.

## Setup

In [1]:
import numpy as np
import pandas as pd
import csv
import os
import sys

import plotly.graph_objs as go

from ostk.mathematics.geometry.d3.objects import Cuboid
from ostk.mathematics.geometry.d3.objects import Composite
from ostk.mathematics.geometry.d3.objects import Point

from ostk.physics.units import Mass
from ostk.physics.units import Length
from ostk.physics.units import Derived
from ostk.physics.units import Angle
from ostk.physics.units import Time
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate import Position
from ostk.physics.coordinate import Velocity
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth
from ostk.physics.environment.objects.celestial_bodies import Sun
from ostk.physics.environment.objects.celestial_bodies import Moon

from ostk.astrodynamics import NumericalSolver
from ostk.astrodynamics.trajectory import State
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.flight.system import SatelliteSystem
from ostk.astrodynamics.flight.system.dynamics import SatelliteDynamics
from ostk.astrodynamics.trajectory.orbit.models import SGP4
from ostk.astrodynamics.trajectory.orbit.models import Propagated
from ostk.astrodynamics.trajectory.orbit.models.sgp4 import TLE

---

## Feeding in reference data

Read in reference data from CSV file 

In [2]:
filename = 'GMAT_EGM96_24hr_30sInterval_run.csv'
plot_title_extension = 'Total position error of a numerically vs analytically propagated orbit'
legend_name_list = ['Numerical Propagation','Analytical Propagation']

multiplication_array = np.array([1.0e3, 1.0e3, 1.0e3])

start_instant = Instant.date_time(DateTime.parse("2022-03-05T13:26:46.742"), Scale.UTC)
frame_gcrf = Frame.GCRF()

with open(f'{os.getcwd()}/Validation Files/{filename}', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    reference_state_array = [] 
    instant_array = []
    for row in reader:
        
        instant_iter = start_instant + Duration.seconds(float(row['YAM.ElapsedSecs']))
        position_iter = Position.meters([row['YAM.EarthICRF.X'], row['YAM.EarthICRF.Y'], row['YAM.EarthICRF.Z']], frame_gcrf)
        velocity_iter = Velocity.meters_per_second([row['YAM.EarthICRF.VX'], row['YAM.EarthICRF.VY'], row['YAM.EarthICRF.VZ']], frame_gcrf)
        
        
        position_iter_m = np.multiply(position_iter.get_coordinates(), multiplication_array)
        velocity_iter_ms = np.multiply(velocity_iter.get_coordinates(), multiplication_array)

        
        instant_array.append(instant_iter)
        reference_state_array.append(State(instant_iter, Position.meters(position_iter_m, frame_gcrf), Velocity.meters_per_second(velocity_iter_ms, frame_gcrf)))

### Propagated Model Setup

Create an environment corresponding to the perturbations that should be used

In [3]:
instant_J2000 = Instant.J2000()
celestial_objects = [Earth.EGM96()]

custom_env = Environment(instant_J2000, celestial_objects)

Create multiple satellite system, a satellite dynamical system, and a numerical solver

In [4]:
mass = Mass(90.0, Mass.Unit.Kilogram)
satellite_geometry = Composite(Cuboid(Point(0.0, 0.0, 0.0), [ [1.0, 0.0, 0.0 ], [ 0.0, 1.0, 0.0 ], [ 0.0, 0.0, 1.0 ] ], [1.0, 0.0, 0.0 ] ))
inertia_tensor = np.ndarray(shape=(3, 3))
surface_area = 0.8
drag_coefficient = 2.2

satellitesystem = SatelliteSystem(mass, satellite_geometry, inertia_tensor, surface_area, drag_coefficient)

start_state = reference_state_array[0]

satellitedynamics = SatelliteDynamics(custom_env, satellitesystem, start_state)

numericalsolver = NumericalSolver(NumericalSolver.LogType.LogAdaptive, NumericalSolver.StepperType.RungeKuttaCashKarp54, 5.0, 1.0e-15, 1.0e-15)

Setup a different Propagated models an an instant array at which propagated states are desired

In [5]:
propagated_model = Propagated(satellitedynamics, numericalsolver)

orbit_propagated = Orbit(propagated_model, custom_env.access_celestial_object_with_name('Earth'))

### TLE and SGP4 Model Setup

In [21]:
tle_end = TLE(
    "1 48915U 21059AN  22089.40584117  .00003779  00000-0  21460-3 0  9997",
    "2 48915  97.5390 219.5716 0011744  13.1851 346.9685 15.13475637 41973"
)

sgp4_model_tle_end = SGP4(tle_end)
orbit_sgp4_tle_end = Orbit(sgp4_model_tle_end, custom_env.access_celestial_object_with_name('Earth'))


gps_instant = Instant.date_time(DateTime.parse("2022-03-30T09:44:24.537"), Scale.UTC)
gps_position = Position.meters([-2837535.43, 6288262.37, 5374.89], Frame.ITRF())
gps_velocity = Velocity.meters_per_second([-1365.64428710937, 621.224060058593, 7541.7861328125], Frame.ITRF())
gps_state_end = State(start_instant, gps_position, gps_velocity)


start_instant = Instant.date_time(DateTime.parse("2022-03-05T13:26:46.742"), Scale.UTC)


print(sgp4_model_tle_end.calculate_state_at(start_instant))

print(gps_state_end.in_frame(Frame.GCRF()))

-- Trajectory :: State -----------------------------------------------------------------------------
    Instant:                                 2022-03-05 13:26:46.742 [UTC]            
    Position:                                [-1036928.693509075558, 652044.607288941508, 6779399.495433156379] [m] @ GCRF 
    Velocity:                                [7273.590213385995, 2036.412223009646, 912.256786916986] [m/s] @ GCRF 
----------------------------------------------------------------------------------------------------

-- Trajectory :: State -----------------------------------------------------------------------------
    Instant:                                 2022-03-30 09:44:24.537 [UTC]            
    Position:                                [256320.761644223705, 6894066.860063605942, 4651.454470396607] [m] @ GCRF 
    Velocity:                                [-1433.296646520525, 1182.801038637040, 7544.832767379140] [m/s] @ GCRF 
--------------------------------------------

In [22]:
tle = TLE(
    "1 48915U 21059AN  22064.61155065  .00002619  00000-0  15045-3 0  9993",
    "2 48915  97.5358 195.0492 0013123  91.5193 268.7541 15.13327131 38229"
)

sgp4_model = SGP4(tle)

orbit_sgp4 = Orbit(sgp4_model, custom_env.access_celestial_object_with_name('Earth'))

desired_instant = Instant.date_time(DateTime.parse("2022-03-05T13:26:46.742"), Scale.UTC)
print(orbit_sgp4.get_state_at(start_instant))



-- Trajectory :: State -----------------------------------------------------------------------------
    Instant:                                 2022-03-05 13:26:46.742 [UTC]            
    Position:                                [-1255462.764318805188, 589751.938079785788, 6746884.937808430754] [m] @ GCRF 
    Velocity:                                [7232.773132820438, 2059.665223864856, 1161.552064619191] [m/s] @ GCRF 
----------------------------------------------------------------------------------------------------



In [ ]:
orbit_list = [orbit_propagated, orbit_sgp4]

desired_instant = Instant.date_time(DateTime.parse("2022-03-05T14:00:00.000"), Scale.UTC)

propagated_state_array_list = [orbit_list[ind].get_state_at(desired_instant) for ind in range(0,len(orbit_list))]

# print(propagated_state_array_list)

Now that everything is set up, we can calculate the state arrays from the desired time instant grid

In [ ]:
orbit_list = [orbit_propagated, orbit_sgp4]

propagated_state_array_list = [orbit_list[ind].get_states_at(instant_array) for ind in range(0,len(orbit_list))]

In [ ]:
def to_dataframe (propagated_state, ind):

    return [
                repr(propagated_state.get_instant()),
                float((propagated_state.get_instant() - propagated_state_array_list[ind][0].get_instant()).in_seconds()),
                *propagated_state.get_position().get_coordinates(),
                *propagated_state.get_velocity().get_coordinates(),
            ]

In [ ]:
propagated_orbit_data_list = [[to_dataframe(propagated_state, ind) for propagated_state in propagated_state_array_list[ind]] for ind in range(0,len(orbit_list))]

In [ ]:
propagated_orbit_df_list = [pd.DataFrame(propagated_orbit_data_list[ind], columns=['$Time^{UTC}$', 'Elapsed secs', '$x_{x}^{ECI}$', '$x_{y}^{ECI}$', '$x_{z}^{ECI}$', '$v_{x}^{ECI}$', '$v_{y}^{ECI}$', '$v_{z}^{ECI}$']) for ind in range(0,len(propagated_model_list))]

Table:

In [ ]:
[propagated_orbit_df_list[ind].head() for ind in range(0,len(orbit_list))]

In [ ]:
[propagated_orbit_df_list[ind].tail() for ind in range(0,len(orbit_list))]

#### Trajectory RMS error vs GMAT run in the GCRF frame 

In [ ]:
def to_dataframe_RMS (ref_ind, list_ind):
    
    return [
                repr(propagated_state_array_list[list_ind][ref_ind].get_instant()),
                
                float((propagated_state_array_list[list_ind][ref_ind].get_instant() - propagated_state_array_list[list_ind][0].get_instant()).in_seconds()),

                (np.linalg.norm((propagated_state_array_list[list_ind][ref_ind].get_position().get_coordinates() - reference_state_array[ref_ind].get_position().get_coordinates()))),
                
                # print(f'Prop:{(propagated_state_array[ref_ind].in_frame(frame_LVLH).get_position().get_coordinates())}'),
                # print(f'Ref:{(reference_state_array[ref_ind].in_frame(frame_LVLH).get_position().get_coordinates())}'),
                (np.linalg.norm((propagated_state_array_list[list_ind][ref_ind].get_velocity().get_coordinates() - reference_state_array[ref_ind].get_velocity().get_coordinates()))),
            ]

In [ ]:
orbit_data_RMS_list = [[to_dataframe_RMS(ref_ind, list_ind) for ref_ind in range(0,len(propagated_state_array_list[list_ind]))]  for list_ind in range(0,len(orbit_list))]

In [ ]:
orbit_df_RMS_list = [pd.DataFrame(orbit_data_RMS_list[list_ind], columns=['$Time^{UTC}$', 'Elapsed secs', '${\delta}x$', '${\delta}v$'])  for list_ind in range(0,len(orbit_list))]

In [ ]:
[orbit_df_RMS_list[list_ind].head() for list_ind in range(0,len(orbit_list))]

In [ ]:
[orbit_df_RMS_list[list_ind].tail() for list_ind in range(0,len(orbit_list))]

# Validation Plots 

Plot position error 

In [ ]:
orbit_df_RMS_position_list = [orbit_df_RMS_list[list_ind][['Elapsed secs', '${\delta}x$']] for list_ind in range(0,len(orbit_list))]

figure = go.Figure()
figure.update_layout(title=f'Position {plot_title_extension}', showlegend = True, height=1000)
figure.update_xaxes(title_text="Time Elapsed (s)")
figure.update_yaxes(title_text="Position Difference in RTN (m)")

for list_ind, orbit_df_RMS_position  in enumerate(orbit_df_RMS_position_list):
    figure.add_trace(go.Scatter(
            x = orbit_df_RMS_position['Elapsed secs'],
            y = orbit_df_RMS_position['${\delta}x$'],
            name = legend_name_list[list_ind],
            mode = 'lines'
        ))
    
figure.show()


Plot velocity error

In [ ]:
orbit_df_RMS_velocity_list = [orbit_df_RMS_list[list_ind][['Elapsed secs', '${\delta}v$']] for list_ind in range(0,len(orbit_list))]

figure = go.Figure()
figure.update_layout(title=f'Velocity {plot_title_extension}', showlegend = True, height=1000)
figure.update_xaxes(title_text="Time Elapsed (s)")
figure.update_yaxes(title_text="Velocity Difference in RTN (m/s)")

for list_ind, orbit_df_RMS_velocity in enumerate(orbit_df_RMS_velocity_list):
    figure.add_trace(go.Scatter(
            x = orbit_df_RMS_velocity['Elapsed secs'],
            y = orbit_df_RMS_velocity['${\delta}v$'],
            name = legend_name_list[list_ind],
            mode = 'lines'
        ))
    
figure.show()
